# B站：神奇的布欧
# 微信：l1243278923

In [1]:
import torch
import torch.nn as nn
import torch.utils.data as Data
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import  train_test_split

# 0. 判断GPU是否可用

In [2]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# 1. 数据预处理

In [3]:
dataPath = "./boston.csv"
df = pd.read_csv(dataPath)

In [4]:
df.shape

(506, 13)

In [5]:
x_data = df.iloc[:, :12].values
y_data = df.MEDV.values
print(x_data.shape)
print(y_data.shape)

(506, 12)
(506,)


In [6]:
# 数据标准化
scaler = StandardScaler()
scaler.fit(x_data)
x_data = scaler.transform(x_data)

In [7]:
# 数据集划分
X_train, X_test, y_train, y_test = train_test_split(x_data, y_data, test_size=0.1, random_state=56)

In [8]:
# 转换为张量
train_xt = torch.from_numpy(X_train.astype(np.float32))
train_yt = torch.from_numpy(y_train.astype(np.float32))
test_xt = torch.from_numpy(X_test.astype(np.float32))
test_yt = torch.from_numpy(y_test.astype(np.float32))

In [9]:
# 数据集加载器
train_data = Data.TensorDataset(train_xt, train_yt)

In [10]:
train_loader = Data.DataLoader(dataset=train_data,
                               batch_size=32,
                               shuffle=True,
                               drop_last=True)

# 2. 网络模型搭建

In [11]:
class Model(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc = nn.Sequential(
            nn.Linear(in_features=12, out_features=64),
            nn.ReLU(),
            nn.Linear(in_features=64, out_features=32),
            nn.ReLU(),
            nn.Linear(in_features=32, out_features=16),
            nn.ReLU(),
            nn.Linear(in_features=16, out_features=1)
        )
    def forward(self, x):
        return self.fc(x)

In [12]:
model = Model().to(device)

In [13]:
model

Model(
  (fc): Sequential(
    (0): Linear(in_features=12, out_features=64, bias=True)
    (1): ReLU()
    (2): Linear(in_features=64, out_features=32, bias=True)
    (3): ReLU()
    (4): Linear(in_features=32, out_features=16, bias=True)
    (5): ReLU()
    (6): Linear(in_features=16, out_features=1, bias=True)
  )
)

# 3. 训练

In [16]:
def train():
    optimizer = torch.optim.SGD(model.parameters(), lr=1e-4)
    loss_fun = nn.MSELoss()
    model.train()
    for epoch in range(100):
        for step, (x, y) in enumerate(train_data):
            out = model(x.to(device))
            loss = loss_fun(torch.squeeze(out.to("cpu")), y)
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()
        if epoch % 5 == 0:
            print(f"epoch:{epoch}, 损失：{loss.item()}")
    torch.save(model, "./6.波士顿房价预测.model")

In [17]:
train()

epoch:0, 损失：3.820988178253174
epoch:5, 损失：0.6058003306388855
epoch:10, 损失：0.8992898464202881
epoch:15, 损失：0.775285005569458
epoch:20, 损失：0.2938010096549988
epoch:25, 损失：0.09228436648845673
epoch:30, 损失：0.0462874174118042
epoch:35, 损失：0.014591722749173641
epoch:40, 损失：0.07201102375984192
epoch:45, 损失：0.1480240374803543
epoch:50, 损失：0.1658148318529129
epoch:55, 损失：0.42741578817367554
epoch:60, 损失：0.37530019879341125
epoch:65, 损失：0.6251446008682251
epoch:70, 损失：0.5618650317192078
epoch:75, 损失：0.7425040006637573
epoch:80, 损失：0.593417763710022
epoch:85, 损失：0.6619986295700073
epoch:90, 损失：0.7514452934265137
epoch:95, 损失：1.129335880279541


In [18]:
@torch.no_grad()
def test(test_xt):
    model = torch.load("./6.波士顿房价预测.model").to("cpu")
    model.eval()
    out = model(test_xt)
    return out

In [19]:
predit = test(test_xt)

In [23]:
predit[6], test_yt[6]

(tensor([9.8906]), tensor(7.2000))